# Svenskt Kvinnobiografiskt lexikon 
version 1.3.2

* Denna [Jupityr Notebook](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon.ipynb) 
* Wikidataproperty [P4963](https://www.wikidata.org/wiki/Property_talk:P4963) 
  * Wikidata ung. 1200 kopplingar SKBL 7/04/2020
* SKBL databas [KARP](https://ws.spraakbanken.gu.se/ws/karp)
  * 1495 poster 7/04/2020
  * 1512 poster 16/08/2020
  
### TODO
* prata ihop oss med SKBL hur vi kan enklare använda varandras data
  * församlingar skrivs idag ut i text i JSON filen med koordinat vore tydligare om ni hade ett Qnummer med exempel
    * Borrby församling visas idag med text och 
    place: { parish: "Borrby församling", pin: { lat: 55.456944, lon: 14.180278 }.....
    * vore tydligare om ni hade med
        * Wikipedia namnet 
        * Wikipedia Q numret
        * se [video](https://www.youtube.com/watch?v=qwVL0OOEdkY)
  * namnet blir idag lite grötigt med "/vera/" skapa hellre nya fält
  * kontakter anges idag som textsträngar bättre med samma som ISNI, Wikidata Qnummer eller annat så vi kan enklare vara säkra på rätt person
  
## Statistik
* idag länkar sv:Wikipedia se [video](https://www.youtube.com/watch?v=qwVL0OOEdkY)
  * med mall SKBL --> 358 artiklar --> 2019 antal visningar > [700 000](https://pageviews.toolforge.org/massviews/?platform=all-access&agent=user&source=category&range=this-year&subjectpage=0&subcategories=0&sort=views&direction=1&view=list&target=https://sv.wikipedia.org/wiki/Kategori:SKBL)
  * med mall Auktioritetsdata --> 1166 artiklar --> 2019 antal visningar > [3,3 miljoner](https://pageviews.toolforge.org/massviews/?platform=all-access&agent=user&source=category&range=this-year&subjectpage=0&subcategories=0&sort=views&direction=1&view=list&target=https://sv.wikipedia.org/wiki/Kategori:Wikipediaartiklar_med_identifierare_fr%25C3%25A5n_SKBL)
  * vi borde kunna lyfta detta vidare och ha samma som för orter/församlingar/organisationer/skolor
  
### sv:Wikipedia     
* SKBL artiklar som [saknar motsvarande artikel på sv:Wikipedia](https://w.wiki/Zui) 
* SKBL artiklar som [saknar Auktoritetsdatamallen](https://petscan.wmflabs.org/?psid=17130486)

### en:Wikipedia    
en:WIkipedia är en av världens 10 största siter så SKBLs engelska artiklar borde försöka länkas därifrån se [länk visningar](https://stats.wikimedia.org/v2/#/en.wikipedia.org)
* SKBL kopplade poster som [har en:Wikipedia artikel](https://w.wiki/Zur) = 534 stycken 
  * som [saknar en:Wikipedia artikel](https://w.wiki/Zup)
* borde tas fram en [SKBL mall](https://sv.wikipedia.org/wiki/Mall:SKBL) på engelsk Wikipedia som pekar på SKBLs engelska artikel
* borde försöka få med SKBL i en:Wikipedia motsvarighet till Auktoritetsdata [Template:Authority_control](https://en.wikipedia.org/wiki/Template:Authority_control)

### ru:Wikipedia
video om hur [Wikipedia och olika språkversioner fungerar](https://youtu.be/GzJfXsdmfeU)
ryska WIkipedia har varit rel enkla att lyfta in Wikidata egenskaper dom har idag [länk visningar](https://stats.wikimedia.org/v2/#/ru.wikipedia.org) 806 M visningar per månad så det kan vara en utmaning
* SKBL kopplade poster som [har ru:Wikipedia artikel](https://w.wiki/Zus) = 568 stycken samma men med [länk ru artikeln](https://w.wiki/Zuz)
  * som [saknar ru:Wikipedia artikel](https://w.wiki/Zup)
* Auktoritetsdata på ru:Wikipedia [Шаблон:Внешние ссылки](https://ru.wikipedia.org/wiki/%D0%A8%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD:%D0%92%D0%BD%D0%B5%D1%88%D0%BD%D0%B8%D0%B5_%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8)


In [231]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None)
pd.set_option('display.max_rows', None)

dftot = pd.DataFrame()
 
url = "https://ws.spraakbanken.gu.se/ws/karp/v5/minientry?q=extended%7C%7Cand%7Cnamn%7Cexists&mode=skbllinks&show=name%2Curl&sort=sorteringsnamn.sort%2Csorteringsnamn.init%2Ctilltalsnamn.sort&size=10000&resource=skbl"
r = http.request('GET', url) 
data = json.loads(r.data.decode('utf-8')) 
#print(type(data))
#print(r.info())
listSKBL = []
for row in (data["hits"]["hits"]):
    #print (row["_source"]["url"])
    listSKBL.append(row["_source"]["url"])
#dftot = pd.DataFrame(listSKBL)
SKBLtot = pd.DataFrame(listSKBL,
                  columns=['skblid'])


In [232]:
SKBLtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513 entries, 0 to 1512
Data columns (total 1 columns):
skblid    1513 non-null object
dtypes: object(1)
memory usage: 11.9+ KB


# Wikidata

In [233]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT * WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P4963 ?skbl
} order by ?skbl"""


def get_results(endpoint_url, query):
    user_agent = "User:salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)
 
lst = []
for result in results["results"]["bindings"]:
    lst.append((result["skbl"]["value"]))
WDtot = pd.DataFrame(lst,columns={'skblid'}) 
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1263 entries, 0 to 1262
Data columns (total 1 columns):
skblid    1263 non-null object
dtypes: object(1)
memory usage: 10.0+ KB


In [234]:
WDtot.sort_index(inplace=True)  

In [235]:
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1263 entries, 0 to 1262
Data columns (total 1 columns):
skblid    1263 non-null object
dtypes: object(1)
memory usage: 10.0+ KB


In [236]:
SKBLtot.drop_duplicates(subset ="skblid", keep = False, inplace = True)
SKBLtot.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384 entries, 0 to 1512
Data columns (total 1 columns):
skblid    1384 non-null object
dtypes: object(1)
memory usage: 21.6+ KB


In [237]:
dfmerge = pd.merge(WDtot, SKBLtot,how='outer', indicator=True)

In [238]:
# check whatsnew / deleted  
dfmerge['_merge'].value_counts()

both          1202
right_only     182
left_only       61
Name: _merge, dtype: int64

## Create OpenRefine file for rights_only
firstname lastname Name , birth death 

In [239]:
SKBLnew = dfmerge[dfmerge['_merge']=="right_only"]


In [240]:
SKBLnew.columns

Index(['skblid', '_merge'], dtype='object')

In [241]:
SKBLnew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1263 to 1444
Data columns (total 2 columns):
skblid    182 non-null object
_merge    182 non-null category
dtypes: category(1), object(1)
memory usage: 3.1+ KB


In [242]:
import csv  
jsonURL = "https://skbl.se/sv/artikel/"
listNewItems =[]
for index,row in SKBLnew.iterrows():
    url = jsonURL + row["skblid"] + ".json"
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))  
    
    new_item = dict()
    try:
        birthdate = data['lifespan']['from']['date']['date'],
    except:
        birtdate =""
    new_item['birthdate'] = birthdate
    try:
        deathdate = data['lifespan']['to']['date']['date']
    except:
        deathdate=""
    new_item['deathdate'] = deathdate
    try:
        birthparish = data['lifespan']['from']['place']['parish']
    except:
        birthparish = ""        
    new_item['birthparish'] = birthparish
    try:
        deathparish = data['lifespan']['to']['place']['parish']
    except:
        deathparish = ""
    new_item['deathparish'] = deathparish
    try:
        ISNI = data['id']['id']
    except:
        ISNI = ""
    new_item['ISNI'] = ISNI
    new_item['SKBL'] = row["skblid"]
    new_item['index'] = index
    new_item['skbl_status'] = data['skbl_status']
    new_item['firstname'] = data['name'].get('firstname')
    new_item['lastname'] = data['name'].get('lastname')
    new_item['subtitle'] = data['subtitle']
    listNewItems.append(new_item)

keys = listNewItems[0].keys()
with open("SKBL_newfile.csv", "w", newline='') as SKBLfile:
    dict_writer = csv.DictWriter(SKBLfile, keys)
    dict_writer.writeheader()
    dict_writer.writerows(listNewItems)

            
    